# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from jorbit.utils.doubledouble import DoubleDouble

In [3]:
a = DoubleDouble(1.0)
b = DoubleDouble(1e20)

a / b

DoubleDouble(1e-20, 5.484672854579043e-37)

In [9]:
from decimal import Decimal, getcontext

# Set precision high enough to capture all digits
getcontext().prec = 40

DoubleDouble.from_string(str(Decimal(1) / Decimal(7))[:32])

DoubleDouble(0.14285714285714285, 7.142857142857e-18)

In [10]:
2**27 134217729

134217728

In [2]:
from jorbit.integrators.ias15_dd import step

TypeError: unsupported operand type(s) for *: 'float' and 'DoubleDouble'

In [5]:
a = DoubleDouble(2.0)
4 * a

TypeError: unsupported operand type(s) for *: 'int' and 'DoubleDouble'

In [ ]:
x = (
    x0
    + (
        (
            (
                (
                    (
                        (
                            (
                                (b.p6 * 7.0 * IAS15_H[n] / 9.0 + b.p5)
                                * 3.0
                                * IAS15_H[n]
                                / 4.0
                                + b.p4
                            )
                            * 5.0
                            * IAS15_H[n]
                            / 7.0
                            + b.p3
                        )
                        * 2.0
                        * IAS15_H[n]
                        / 3.0
                        + b.p2
                    )
                    * 3.0
                    * IAS15_H[n]
                    / 5.0
                    + b.p1
                )
                * IAS15_H[n]
                / 2.0
                + b.p0
            )
            * IAS15_H[n]
            / 3.0
            + a0
        )
        * dt
        * IAS15_H[n]
        / 2.0
        + v0
    )
    * dt
    * IAS15_H[n]
)
v = (
    v0
    + (
        (
            (
                (
                    (
                        (
                            (b.p6 * 7.0 * IAS15_H[n] / 8.0 + b.p5)
                            * 6.0
                            * IAS15_H[n]
                            / 7.0
                            + b.p4
                        )
                        * 5.0
                        * IAS15_H[n]
                        / 6.0
                        + b.p3
                    )
                    * 4.0
                    * IAS15_H[n]
                    / 5.0
                    + b.p2
                )
                * 3.0
                * IAS15_H[n]
                / 4.0
                + b.p1
            )
            * 2.0
            * IAS15_H[n]
            / 3.0
            + b.p0
        )
        * IAS15_H[n]
        / 2.0
        + a0
    )
    * dt
    * IAS15_H[n]
)

In [ ]:
H1AX = 7.0 * IAS15_H1 / 9.0
H1BX = 3.0 * IAS15_H1 / 4.0
H1CX = 5.0 * IAS15_H1 / 7.0
H1DX = 2.0 * IAS15_H1 / 3.0
H1EX = 3.0 * IAS15_H1 / 5.0
H1FX = IAS15_H1 / 2.0
H1GX = IAS15_H1 / 3.0

H1AV = 7.0 * IAS15_H1 / 8.0
H1BV = 6.0 * IAS15_H1 / 7.0
H1CV = 5.0 * IAS15_H1 / 6.0
H1DV = 4.0 * IAS15_H1 / 5.0
H1EV = 3.0 * IAS15_H1 / 4.0
H1FV = 2.0 * IAS15_H1 / 3.0

x = (
    x0
    + (
        (
            (
                (
                    ((((b.p6 * H1AX + b.p5) * H1BX + b.p4) * H1CX + b.p3) * H1DX + b.p2)
                    * H1EX
                    + b.p1
                )
                * H1FX
                + b.p0
            )
            * +a0
        )
        * dt
        * H1FX
        + v0
    )
    * dt
    * IAS15_H1
)
v = (
    v0
    + (
        (
            (
                ((((b.p6 * H1AV + b.p5) * H1BV + b.p4) * H1CV + b.p3) * H1DV + b.p2)
                * H1EV
                + b.p1
            )
            * H1FV
            + b.p0
        )
        * H1FX
        + a0
    )
    * dt
    * IAS15_H1
)

In [12]:
def twosum(a, b):
    s = a + b
    v = s - a
    e = (a - (s - v)) + (b - v)
    return s, e


def addtwo(a, b):
    r = a.hi + b.hi
    s = jnp.where(
        jnp.abs(a.hi + a.lo) > jnp.abs(b.hi + b.lo),
        a.hi - r + b.hi + b.lo + a.lo,
        b.hi - r + a.hi + a.lo + b.lo,
    )
    z = r + s
    zz = r - z + s

    # # High part of the sum
    # s = a.hi + b.hi

    # # Error terms
    # v = s - a.hi
    # t = ((b.hi - v) + (a.hi - (s - v))) + a.lo + b.lo
    return z, zz


addtwo(DoubleDouble(1e20), DoubleDouble(1.0))

(Array(1.e+20, dtype=float64, weak_type=True),
 Array(1., dtype=float64, weak_type=True))

In [14]:
2**-28

3.725290298461914e-09